###チェーンとは
LLM は単独でも十分に強力に機能します。 しかし、 LLM 同士を組合わせたり、ある機能に特化した他のモジュールとともに利用することで、より複雑なアプリケーションを構築することができます。 LangChain では、このような他の機能と連結するための汎用的なインターフェースとして、チェーンを提供しています。 チェーンを用いることで、LLM の利用を含む "一連の処理" を一つのまとまりとして扱うことができます。<br><br>

つまり、平易な言葉でいえば、チェーンは「複数の処理の連なり」です。 この処理の連鎖の部品となるチェーンの構成要素のことを リンク と呼びます。 リンクの一例は、LLM の呼び出しなどの基本的な処理です。さらには、その他のチェーン全体をリンクとして含むチェーンも作成できます。<br><br>

チェーンの代表例は、LLM とプロンプトテンプレートを組合わせて使用するための LLMChain です。 このチェーンを用いると、<br><br>

ユーザーの入力を受け取り<br>
それをPromptTemplateでフォーマットし<br>
フォーマットされたレスポンスを LLM に渡す<br><br>
という一連の操作を一つのまとまりとして実行できます。 このチェーンの使用方法については、 「LLM とプロンプトテンプレートを組み合わせて使用する」や 「チェーンをチャットモデルと組合わせて使用する」で解説しています。<br><br>

このページでは、チェーンの使用例として以下のケースを紹介します。<br><br>

SequentialChain を用いた複数チェーンの連結<br>
Chain クラスを用いたカスタムチェーンの作成<br><br>
その他のチェーンに関しても知りたい方は、LangChain 公式ドキュメント | Chains | How-To Guide や LangChain 公式ドキュメント | Chains | Reference を参照してください。<br><br>

チェーンの使用方法の概要は以下の通りです。<br><br>

チェーンを作成する<br>
チェーンにコンポーネントを追加する<br>
チェーンを実行する<br>
このページでは、これらの手順を Python コードとともに解説します。<br><br>

SequentialChain を用いた複数チェーンの連結<br>
SequentialChain は、複数のチェーンを連結するためのチェーンです。これは、あらかじめ定義された順番に沿ってリンクを実行します。 このチェーンを用いることで、例えば、言語モデルを呼び出した後に、さらに連続して言語モデルを呼び出すことができます。
<br><br>
では、SimpleSequentialChain の使用例をみてみましょう。 このチェーンは<br><br>

各ステップの入出力が一つ<br>
各ステップの出力が次のステップの入力になる<br>
という特徴を持つもっとも単純なタイプの SequentialChain です。<br>

In [ ]:
pip install google-search-results
pip install openai
pip install langchain

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv(OPENAI_API_KEY)
os.environ["SERPAPI_API_KEY"] =os.getenv(SERPAPI_API_KEY)

In [8]:
# llm ラッパーのインポート
from langchain import OpenAI
# プロンプトテンプレートのインポート
from langchain.prompts import PromptTemplate

# LLMChain に加えて SimpleSequentialChain もインポートする
# SimpleSequentialChain は、複数のチェーンを連続実行するためのチェーンで以下の特徴をもつ
# - 各ステップの入出力は一つ
# - 各ステップの出力が次のステップの入力になる
from langchain.chains import LLMChain, SimpleSequentialChain

# llm ラッパーの初期化
llm = OpenAI(temperature=0)

# 最初のプロンプトテンプレートの作成
prompt_first = PromptTemplate(
    input_variables=["product"],
    template="{product}を作る会社の社名として、何かいいものはないですか？日本語の社名でお願いします。",
)

# 最初に実行する LLM チェーンを定義
# 会社名を考えてもらう
chain_first = LLMChain(llm=llm, prompt=prompt_first)

# 次のプロンプトテンプレートの作成
prompt_second = PromptTemplate(
    input_variables=["company_name"],
    template="{company_name}という会社名の企業のキャッチコピーを考えてください。",
)

# 次に実行する LLM チェーンを定義
# キャッチコピーを考えてもらう
chain_second = LLMChain(llm=llm, prompt=prompt_second)

# 二つの LLM チェーンを連結
overall_chain = SimpleSequentialChain(chains=[chain_first, chain_second], verbose=True)

# 連結してできたチェーンを実行
chatchphrase = prediction = overall_chain.run("カラフルな靴下")
print(chatchphrase)



> Entering new SimpleSequentialChain chain...


「カラフルソックス工房」


「足元を彩る、カラフルソックス工房」

> Finished chain.


「足元を彩る、カラフルソックス工房」


###Chain クラスを用いたカスタムチェーンの作成
LangChain には多くのチェーンが用意されていますが、時には特定のユースケースのためにカスタムチェーンを作りたい場合もあるでしょう。 このような要望は Chain クラスを使用すれば達成できます。

カスタムクラスを作成するには、以下の手順を踏みます。

Class のサブクラスを作成
input_keys と output_keys を定義
_call メソッドを定義（チェーンの実行方法を定義する）
以下に Chain クラスを使用して、２つの LLMChain の出力を連結するカスタムチェーンを作成する例を紹介します。

In [10]:
from typing import Dict, List

# LLMChain のインポート
from langchain.chains import LLMChain

# カスタムチェーンを定義するために Chain クラスをインポート
from langchain.chains.base import Chain


# このチェーンは、２つの LLMChain の出力を連結する
class ConcatenateChain(Chain):
    # もととなる２つの LLMChain をコンストラクタで受け取る
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # ２つのチェーンの入力キーの和集合
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        # このチェーンの出力キーは、concat_output のみ
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        # `output_keys` で定義したキーをもつ Dict を返す関数を定義する
        # ここでは、２つのチェーンを独立に実行した得られた出力を連結して返す
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [11]:
# llm ラッパーのインポート
from langchain import OpenAI
# LLM チェーンの構築で必要なプロンプトテンプレートのインポート
from langchain.prompts import PromptTemplate

# llm ラッパーの初期化
llm = OpenAI(temperature=0)

prompt_1 = PromptTemplate(
    input_variables=["product"],
    template="{product}を作る会社の社名として、何かいいものはないですか？日本語の社名でお願いします。",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["product"],
    template="{product}を作る会社のスローガンとして、何かいいものはないですか？日本語でお願いします。",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("カラフルな靴下")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:


「カラフルソックス工房」

「足元を彩る、カラフルな靴下」
